In [1]:
import numpy as np
from scipy.integrate import odeint

def make_var_stationary(beta, radius=0.97):
    '''Rescale coefficients of VAR model to make stable.'''
    p = beta.shape[0]
    lag = beta.shape[1] // p
    bottom = np.hstack((np.eye(p * (lag - 1)), np.zeros((p * (lag - 1), p))))
    beta_tilde = np.vstack((beta, bottom))
    eigvals = np.linalg.eigvals(beta_tilde)
    max_eig = max(np.abs(eigvals))
    nonstationary = max_eig > radius
    if nonstationary:
        return make_var_stationary(0.95 * beta, radius)
    else:
        return beta

def simulate_var(p, T, lag, sparsity=0.2, beta_value=1.0, sd=0.1, seed=0):
    if seed is not None:
        np.random.seed(seed)

    # Set up coefficients and Granger causality ground truth.
    GC = np.eye(p, dtype=int)
    beta = np.eye(p) * beta_value

    num_nonzero = int(p * sparsity) - 1
    for i in range(p):
        choice = np.random.choice(p - 1, size=num_nonzero, replace=False)
        choice[choice >= i] += 1
        beta[i, choice] = beta_value
        GC[i, choice] = 1

    beta = np.hstack([beta for _ in range(lag)])
    beta = make_var_stationary(beta)

    # Generate data.
    burn_in = 100
    errors = np.random.normal(scale=sd, size=(p, T + burn_in))
    X = np.zeros((p, T + burn_in))
    X[:, :lag] = errors[:, :lag]
    for t in range(lag, T + burn_in):
        X[:, t] = np.dot(beta, X[:, (t-lag):t].flatten(order='F'))
        X[:, t] += errors[:, t-1]

    return X.T[burn_in:], beta, GC

def lorenz(x, t, F):
    '''Partial derivatives for Lorenz-96 ODE.'''
    p = len(x)
    dxdt = np.zeros(p)
    for i in range(p):
        dxdt[i] = (x[(i+1) % p] - x[(i-2) % p]) * x[(i-1) % p] - x[i] + F

    return dxdt

def simulate_lorenz_96(p, T, F=10.0, delta_t=0.1, sd=0.1, burn_in=1000, seed=0):
    if seed is not None:
        np.random.seed(seed)

    # Use scipy to solve ODE.
    x0 = np.random.normal(scale=0.01, size=p)
    t = np.linspace(0, (T + burn_in) * delta_t, T + burn_in)
    X = odeint(lorenz, x0, t, args=(F,))
    X += np.random.normal(scale=sd, size=(T + burn_in, p))

    # Set up Granger causality ground truth.
    GC = np.zeros((p, p), dtype=int)
    for i in range(p):
        GC[i, i] = 1
        GC[i, (i + 1) % p] = 1
        GC[i, (i - 1) % p] = 1
        GC[i, (i - 2) % p] = 1

    return X[burn_in:], GC

# Parameters for simulation
T_values = [200, 500, 1000]
seeds = [169, 381, 584, 594, 618, 671, 981]
p = 5  # number of variables
lag = 5  # VAR lag
sparsity = 0.2
beta_value = 1.0
sd = 0.1
F = 10.0
delta_t = 0.1
burn_in = 1000

# Simulating VAR and Lorenz models and saving results
for T in T_values:
    for seed in seeds:
        # Simulate VAR model
        X_var, beta, GC_var = simulate_var(p, T, lag, sparsity=sparsity, beta_value=beta_value, sd=sd, seed=seed)
        np.savez(f'results_var_T_{T}_seed_{seed}.npz', 
                 X_np=X_var, 
                 Gref=GC_var, 
                 n=p, 
                 T=T, 
                 sparsity=sparsity, 
                 var_lag=lag, 
                 sd=sd, 
                 beta_value=beta_value, 
                 seed=seed)

        # Simulate Lorenz model
        X_lorenz, GC_lorenz = simulate_lorenz_96(p, T, F=F, delta_t=delta_t, sd=sd, burn_in=burn_in, seed=seed)
        np.savez(f'results_lorenz_T_{T}_seed_{seed}.npz', 
                 X_np=X_lorenz, 
                 Gref=GC_lorenz, 
                 n=p, 
                 T=T, 
                 F=F, 
                 delta_t=delta_t, 
                 sd=sd, 
                 burn_in=burn_in, 
                 seed=seed)
